# Baseline
### Pick a. small instruct model, build deterministic parsing prompt + JSON Schema Validator

# SFT (Supervised Fine-Tuning) for synthetic Dataset
### Train with Hugging Face TRL SFTTrainer, Use PEFT LoRA adapters so training is light

# Evaluate LLM

### JSON validity rate, Field level accuracy, business metric % events created without manual edits

# DPO
### Generate Pairs (good parse vs bad parse) from your synthetic edge cases, Train with TRL's DPO style pipeline